In [5]:
import pandas as pd
import numpy as np

In [8]:
#load file prices & convert updated_at to datetime object (using pd.to_datetime)
prices = pd.read_csv('prices.csv')
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
prices = prices.sort_values(by='updated_at')
prices

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


In [9]:
#load file sales & convert ordered_at to datetime object (using pd.to_datetime)
sales = pd.read_csv('sales.csv')
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
sales = sales.sort_values(by='ordered_at')
sales

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


In [10]:
# find the "nearest" update timestamp of each order
merge_nearest = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')

In [11]:
# compare ordered_at and "nearest" updated_at of each order to find the correct price
merge_nearest['listed_price'] = np.where(merge_nearest['ordered_at'] >= merge_nearest['updated_at'],
                                         merge_nearest['new_price'], merge_nearest['old_price'])
merge_nearest.head(10)

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,listed_price
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
5,4085861,2018-09-11 11:06:00,1,60000,53500,2018-09-11 08:51:00,53500
6,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
7,3954203,2018-09-11 11:11:00,1,60000,64000,2018-09-11 11:54:00,60000
8,4085861,2018-09-11 11:34:00,1,60000,53500,2018-09-11 08:51:00,53500
9,4085861,2018-09-11 11:47:00,2,60000,53500,2018-09-11 08:51:00,53500


In [12]:
# calculate total revenue for each product
merge_nearest['revenue'] = merge_nearest['quantity_ordered'] * merge_nearest['listed_price']
total_revenue_by_product_and_price = merge_nearest.groupby(['product_id', 'listed_price'], as_index=False)['revenue'].sum()
total_revenue_by_product_and_price

,product_id,listed_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [13]:
total_revenue = merge_nearest.groupby('product_id', as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500
